In [1]:
import geopandas as gpd
import cdsapi
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import rioxarray
from rasterio.features import geometry_mask

In [2]:
IF_DOWNLOAD=False
LOCATION_PATH='locations/chn_admbnda_adm2_ocha_2020.shp'

In [3]:
def query_data(years, dataset, request):

    for year in years:
        request['year']=year
        client = cdsapi.Client()
        client.retrieve(dataset, request, target=f"cloud_cover/era5_cloudcover_{year}.nc")

if IF_DOWNLOAD:
    years=[str(year) for year in range(1990, 2025)]
    dataset = "derived-era5-single-levels-daily-statistics"
    request = {
        "product_type": "reanalysis",
        "variable": ["total_cloud_cover"],
        "month": ["08", "09", "10"],
        "day": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11", "12",
            "13", "14", "15",
            "16", "17", "18",
            "19", "20", "21",
            "22", "23", "24",
            "25", "26", "27",
            "28", "29", "30",
            "31"
        ],
        "daily_statistic": "daily_mean",
        "time_zone": "utc+00:00",
        "frequency": "6_hourly",
        "area": [20.5, 108.5, 18, 111.5]
    }

    query_data(years, dataset, request)


In [4]:
hainan=gpd.read_file(LOCATION_PATH)
hainan=hainan[hainan['ADM1_PCODE']=='CN046']
hainan.drop(columns=['OBJECTID', 'Admin_type', 'Adm2_CAP', 'ADM2_PCODE', 'ADM1_ZH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_ZH', 'ADM0_PCODE'], inplace=True)

In [ ]:
hainan.plot(edgecolor="black", facecolor="lightblue")
plt.title("Hainan Map")
plt.show()

In [ ]:
date="2024-09-21"

proj=ccrs.PlateCarree()

fig, ax=plt.subplots(figsize=(10,8), subplot_kw={'projection': proj})
hainan = hainan.to_crs("EPSG:4326")
hainan.boundary.plot(ax=ax, edgecolor='red', linewidth=1.5, transform=proj, zorder=2)
ds = xr.open_dataset("cloud_cover/era5_cloudcover_2024.nc")
da_tcc = ds["tcc"]
da_tcc = da_tcc.rio.write_crs("EPSG:4326")
da_day = da_tcc.sel(valid_time=date) 

masked_tcc = da_day.rio.clip(hainan.geometry, hainan.crs, drop=True)

masked_tcc.plot(cmap="viridis", vmin=0, vmax=1)
for _, row in hainan.iterrows():
    city_name=row['ADM2_EN'].split(' ')[0]
    x, y = row.geometry.centroid.x, row.geometry.centroid.y
    ax.text(x,y,city_name, transform=proj, fontsize=8, ha='center', color='white', weight='bold', zorder=3)
plt.title(f"Total Cloud Cover over Hainan ({date})")
plt.show()